# 12_Built-in module

## 12.1 datetime

### 1. 获取当前时间和日期

In [5]:
from datetime import datetime

In [6]:
now = datetime.now()  #获取当前时间

In [7]:
print(now)

2020-04-16 17:24:30.339169


In [8]:
print(type(now))

<class 'datetime.datetime'>


- `datetime`是模块，该模块内还包含一个`datetime`类
- 可换成`import datetime.datetime`

### 2. 获取指定日期和时间

In [9]:
dt = datetime(2015,4,19,12,20,49)
print(dt)

2015-04-19 12:20:49


### 3. datetime和timestamp的转换

在计算机中，时间实际上是用数字表示的。我们把1970年1月1日 00:00:00 UTC+00:00时区的时刻称为epoch time，记为0（1970年以前的时间timestamp为负数），当前时间就是相对于epoch time的秒数，称为timestamp。

timestamp = 0 = 1970-1-1 00:00:00 UTC+0:00  

对应的北京时间是：  
timestamp = 0 = 1970-1-1 08:00:00 UTC+8:00

为什么计算机用timestamp表示当前时间的?  
因为timestamp的值与时区毫无关系，全球各地的计算机在任意时刻的timestamp都是完全相同的。

In [15]:
#datetime -> timestamp
dt = datetime(2015, 4, 19, 12, 20) # 用指定日期时间创建datetime
dt.timestamp() # 把datetime转换为timestamp

1429438800.0

In [16]:
#timestamp -> datetime
t = 1429438800.0  #浮点数，没有时区概念
print(datetime.fromtimestamp(t))

2015-04-19 12:20:00


datetime和timestamp的区别;  
- timestamp是浮点数，没有时区概念  
- datetime是有时区的

In [28]:
t = 1429438800.0  
print(datetime.fromtimestamp(t)) # 本地时间
print(datetime.utcfromtimestamp(t))  #utc时间

2015-04-19 12:20:00
2015-04-19 10:20:00


### 4. str和datetime的转换

In [20]:
cday = datetime.strptime('2015-6-1 18:19:59', '%Y-%m-%d %H:%M:%S')
print(cday)

2015-06-01 18:19:59


注意： 转换后的datetime是没有时区信息的

In [21]:
now = datetime.now()
print(now.strftime('%a, %b %d %H:%M'))

Thu, Apr 16 19:40


### 5. datetime加减

In [22]:
from datetime import timedelta

In [24]:
now = datetime.now()
datetime(2015,5,18,16,57,3,540997)

datetime.datetime(2015, 5, 18, 16, 57, 3, 540997)

In [25]:
now+timedelta(hours=10)

datetime.datetime(2020, 4, 17, 5, 42, 25, 943342)

In [26]:
now - timedelta(days=1)

datetime.datetime(2020, 4, 15, 19, 42, 25, 943342)

In [27]:
now+timedelta(days=2,hours=12)

datetime.datetime(2020, 4, 19, 7, 42, 25, 943342)

### 6. 本地时间和UTC时间的转换 

本地时间： 系统设定时区的时间，例如北京时间是UTC+8:00时区的时间  
UTC时间： UTC+0:00时区的时间

`datetime`类型中有一个时区属性`tzinfo`，默认为`None`，所以无法区分这个`datetime`到底是哪个时区，除非强行给`datetime`设置一个时区

In [42]:
from datetime import datetime, timedelta, timezone
tz_utc_1 = timezone(timedelta(hours=1)) # 创建时区UTC+1:00(德国)
now = datetime.now()

In [43]:
now

datetime.datetime(2020, 4, 16, 20, 5, 29, 472157)

In [44]:
dt = now.replace(tzinfo=tz_utc_1) 
dt

datetime.datetime(2020, 4, 16, 20, 5, 29, 472157, tzinfo=datetime.timezone(datetime.timedelta(0, 3600)))

注意：如果系统时区恰好是UTC+8:00，那么上述代码就是正确的，否则，不能强制设置为UTC+8:00时区。

### 7. 时区转换

可以先通过`utcnow()`拿到当前的UTC时间，再转换为任意时区的时间：

In [45]:
# 拿到UTC时间，并强制设置时区为UTC+0:00:
utc_dt = datetime.utcnow().replace(tzinfo=timezone.utc)
print(utc_dt)

2020-04-16 18:12:18.487158+00:00


In [46]:
#astimezone()将转换时区为北京时间:
bj_dt = utc_dt.astimezone(timezone(timedelta(hours=8)))
print(bj_dt)

2020-04-17 02:12:18.487158+08:00


In [47]:
#UTC时间转为东京时间
tokyo_dt = utc_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt)

2020-04-17 03:12:18.487158+09:00


In [48]:
#bj_dt转换为东京时间
tokyo_dt2 = bj_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt2)

2020-04-17 03:12:18.487158+09:00


- 时区转换的关键在于，拿到一个`datetime`时，要获知其正确的时区，然后强制设置时区，作为基准时间。
- 利用带时区的`datetime`，通过`astimezone()`方法，可以转换到任意时区。
- 不是必须从UTC+0:00时区转换到其他时区，任何带时区的`datetime`都可以正确转换，例如上述`bj_dt`到`tokyo_dt`的转换。
- `datetime`表示的时间需要时区信息才能确定一个特定的时间，否则只能视为本地时间。
- 如果要存储`datetime`，最佳方法是将其转换为`timestamp`再存储，因为timestamp的值与时区完全无关

练习  

假设你获取了用户输入的日期和时间如2015-1-21 9:01:30，以及一个时区信息如UTC+5:00，均是str，请编写一个函数将其转换为timestamp：  

In [59]:
import re
from datetime import datetime, timezone, timedelta
def to_timestamp(dt_str,tz_str):
    dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
    tz = re.match(r'\w{3}([\+\-]\d{1,2})\:\d{2}', tz_str).group(1)
    tz = int(tz)
    t_utc = dt.replace(tzinfo = timezone(timedelta(hours=tz)))
    return t_utc.timestamp()